# search for specific words

In [1]:
# Google Sheets Variables
# variables that shouldn't change by person

SEARCH_WORKBOOK = {'sheet_id' : '1QjvZOnkCJM-BcRvMlP0XsN0hJeanKPiK7_e37uJiKek',
                    'range_names': ['twitter_keywords','stop_phrases']}

RESULTS_WORKBOOK = {'sheet_id' : '1_6-O1D7UtbA4PiDNr71Fm-kA_OJk8S7RBctkGjZRIVk',
                     'range_names': ['tweet_list','staged_tweets','twitter_authors']}




In [2]:
import tweepy
#import pandas as pd
from collections import defaultdict
import re
import json
import requests
import os
from dotenv import load_dotenv


from pprint import pprint
from datetime import datetime
import urllib.parse

#https://www.thepythoncode.com/article/translate-text-in-python
from googletrans import Translator, constants
from pprint import pprint

#to connect to google
from google.oauth2 import service_account
import pygsheets


import pandas as pd
import time


import jmespath
import urlexpander

In [3]:
#User-Specific Variables

#Source of Inputs: GoogleSheets or LocalFile
control_input = 'googlesheets'


#for a continuous loop, put 0 or a negative number.
stop_after=-1
sleeptime = 60*60*10 #10 hour sleep time... to gradually roll to different times of day to catch different time zones.
search_languages = ['en']

In [4]:
load_dotenv()

#twitter variables
API_Key = os.environ.get("API_KEY")
API_Key_Secret = os.environ.get("API_KEY_SECRET")
Bearer_Token = os.environ.get("BEARER_TOKEN")

auth = tweepy.OAuthHandler(API_Key, API_Key_Secret)
bearer = 'bearer ' + Bearer_Token

#google variables
SERVICE_ACCOUNT_FILE = os.environ.get("GOOGLE_SERVICE_ACCOUNT_FILE")

In [5]:
# init the Google API translator
translator = Translator()

runDate = datetime.today().strftime('%Y-%m-%d %H:%M:%S')


In [6]:
def tweetQueryBuilder(searchTerms):
    base_api_url = 'https://api.twitter.com/2/tweets/search/recent?query='
    fixed_args = '-is:retweet  has:links '
    
    query_arg_prefix = fixed_args + searchTerms
    query_arg_prefix = urllib.parse.quote(query_arg_prefix)
    query_arg_suffix = '&max_results=100&tweet.fields=attachments,author_id,context_annotations,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,source,text,withheld&expansions=referenced_tweets.id,geo.place_id'
    
    query = base_api_url + query_arg_prefix + query_arg_suffix
    
    return query


In [7]:
def instantiate_google_sheets_connector():
    # Set up Google Credentials
    #SERVICE_ACCOUNT_FILE = google_service_account_file

    SCOPES = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    # connect
    gc = pygsheets.authorize(credentials=service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, 
        scopes=SCOPES))
    
    return gc

gc = instantiate_google_sheets_connector()

### Load all google sheets

In [8]:
#load to workbook objects
search_workbook = gc.open_by_key(SEARCH_WORKBOOK['sheet_id'])
results_workbook = gc.open_by_key(RESULTS_WORKBOOK['sheet_id'])


In [9]:
#generate read-only dataframes
keywords = search_workbook[0]
#df_r_keywords = pd.DataFrame(keywords.get_all_records())

stop_phrases = search_workbook[1]
#df_r_stop_phrases = pd.DataFrame(stop_phrases.get_all_records())

#generate updatable dataframes
tweet_list = results_workbook[0]
#df_u_tweet_list = pd.DataFrame(tweet_list.get_all_records())

staged_tweets = results_workbook[1]
#df_u_staged_tweets = pd.DataFrame(staged_tweets.get_all_records())

twitter_authors = results_workbook[2]
global DF_AUTHORS
DF_AUTHORS = pd.DataFrame(twitter_authors.get_all_records())

In [10]:
def refresh_queries(control_input):
    
    if control_input == 'googlesheets':
        #google:

        # Set up Google Credentials

        SCOPES = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
        credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        # connect
        gc = pygsheets.authorize(credentials=service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES))
        #sh = gc.open_by_link('https://docs.google.com/spreadsheets/d/1QjvZOnkCJM-BcRvMlP0XsN0hJeanKPiK7_e37uJiKek/edit?usp=sharing')
        workbook = gc.open_by_key(SEARCH_WORKBOOK['sheet_id'])

        keyword_worksheet = workbook[0]
        df_keywords = pd.DataFrame(keyword_worksheet.get_all_records())
        queries = df_keywords['key_words'][(df_keywords['record_type'] == 'search_term') &\
                                           (df_keywords['status'] == 'Active')].values.tolist()
        #print(df_keywords['key_words'][(df_keywords['record_type'] == 'search_term') \
        #                                   (df_keywords['status'])])

    else: #legacy or testing
        queries = [

            'disaster assessment',
            'disaster blockchain',
            'disaster response',
            'disaster openstreetmap',
            'openstreetmap'

        ]
    return queries






In [11]:
def write_results_to_google_sheets(parsed):
    t_keys = []
    t_details = []
    
    nourlcnt=0
    dupcnt=0
    newcnt=0
    urldupcnt=0

    #df_already_saved_tweets = pd.DataFrame(tweet_list.get_all_records()) #seed the list with one row of data
    df_already_saved_tweets = pd.DataFrame(staged_tweets.get_all_records()) #seed the list with one row of data

    for tweet in parsed['data']:
        urls = jmespath.search("entities.urls[*].expanded_url", tweet)
        urls = remove_twitter_urls(urls)
        #print(f'urls are {urls}')
        if len(urls) == 0:
            nourlcnt +=1
            next

        tweet_id = str(jmespath.search("id", tweet))

        #determine the number of times that particular tweet has already been saved
        saved_cnt = df_already_saved_tweets[df_already_saved_tweets['tweet_id'].astype(str) == tweet_id].shape[0]
        
                                               
        if saved_cnt > 0:
            #hard dedupe
            dupcnt+=1
            next

        else:
        #next dedupes will return a dupe-code instead
            tweet_info = extract_tweet_info(tweet)
            if tweet_info[5] == 'NA': #if all urls turn out to be twitter urls, skip
                nourlcnt+=1
                next
            else:
                url_dupe_cnt = df_already_saved_tweets[df_already_saved_tweets['reference_url'] == tweet_info[5]].shape[0]
                if url_dupe_cnt > 0:
                    urldupcnt+=1
                    print(f'{tweet_id} dupeurl {tweet_info[5]}')
                    next
                else:
                    t_details.append(tweet_info)
                    t_keys.append([runDate,tweet_id, 'query_twitter script'])
                    newcnt+=1

    #if anything remains, write to the tweet_list sheet
    if len(t_keys) > 0:
        tweet_list.append_table(t_keys, start='A1', end=None, dimension='ROWS', overwrite=False)
        staged_tweets.append_table(t_details, start='A1', end=None, dimension='ROWS', overwrite=False)
    else:
        print('No new content to add')

    print(f'AT completion of write to google sheets, stats are {dupcnt} tweet id dupes, {nourlcnt} no urls, \
           {urldupcnt} url dupes, {newcnt} new')
    

In [12]:
def remove_twitter_urls(urls):
    
    #capture urls
    patterns = [re.compile("pic\.twitter\.com"), 
                re.compile("twitter\.com")]
    returnUrl = []
    
    for url in urls:
        include = 'Y'
        for pattern in patterns:
            if pattern.search(url) != None:
                include = 'N'
        if include == 'Y':
            returnUrl.append(url)
    
    return returnUrl

In [13]:
def get_author_info(author_id):
    global DF_AUTHORS
    global runDate
    #add the "TID"
    internal_author_id = 'TID' + author_id
    
    auth_list = DF_AUTHORS[DF_AUTHORS['author_id'] == internal_author_id].values.tolist()
    if len(auth_list) == 1:
        where = 'locally'
        author_username = auth_list[0][3]

    elif len(auth_list) > 1:
        author_username = 'UNKN_MultipleResultsReturned'

    else:
        print(f'no individual author match on {author_id} ... trying twitter')
        
        userFields = 'location,url,description,entities'
        api_getAuthor = f'https://api.twitter.com/2/users?ids={author_id}&user.fields={userFields}'
        response = requests.get(api_getAuthor, headers={"Authorization":bearer})
        author_from_twitter = json.loads(response.text)
        time.sleep(2) #throttle volume
        #print(f' found {author_from_twitter}')

        if 'data' in author_from_twitter:
            where = 'twitter'

            author_name = author_from_twitter['data'][0]['name']
            author_username = author_from_twitter['data'][0]['username']
            try:
                author_description = author_from_twitter['data'][0]['description']
            except:
                author_description = ''
            try:
                author_url = author_from_twitter['data'][0]['url']
            except:
                author_url = ''

        elif 'errors' in   author_from_twitter:
            author_name = author_from_twitter['errors'][0]['title']
            author_username = author_from_twitter['errors'][0]['title']
            author_description = author_from_twitter['errors'][0]['detail']
            author_url = author_from_twitter['errors'][0]['type']

        else:
            author_username = 'UNKNN_NoResultsReturned'



        # add values to google sheets
        vals= [runDate, internal_author_id, author_name, author_username, author_description, author_url]
        
        #print(f'before cleanup of {vals}')
        #clean up text 
        for i, item in enumerate(vals):
            vals[i] = scrub_text(item)
        #print(f'after cleanup of {vals}')
            
        twitter_authors.append_table(vals, start='A1', end=None, dimension='ROWS', overwrite=False)
        
        # add values to DF_AUTHORS
        r = pd.Series(vals, index = DF_AUTHORS.columns)
        DF_AUTHORS = DF_AUTHORS.append(r, ignore_index=True)
        
        
    return author_username
    #return [internal_author_id, author_name, author_username, author_description, author_url]
    

assert get_author_info('2576444334') == 'TheMissingMaps' #positive use-case
assert get_author_info('1477759062884421631') == 'Not Found Error' #negative use-case



### Cascade Dupe Checking
1. Hard Dupe on TweetId
2. Soft Dupe on similar tweet text
3. Soft Dupe on target url


In [14]:
def extract_tweet_info(tweet):
    
    tweet_id = str(jmespath.search("id", tweet))
    
    urls = jmespath.search("entities.urls[*].expanded_url", tweet)
    urls = remove_twitter_urls(urls)
            
    text = jmespath.search("text", tweet)
    text = scrub_text(text)
    

    original_lang = jmespath.search("lang", tweet)
    if original_lang == 'en':
        pass
    else:
        try:
            translation = translator.translate(text, src=original_lang, dest='en')
            text = translation.text
        except:
            print(f'language issue {original_lang}')
    

    created_date = jmespath.search("created_at", tweet)
    create_date, create_time = created_date.split('T')
    create_time = create_time[:8]
    
    author_id = jmespath.search("author_id", tweet)
    
    
    
    author_username = get_author_info(author_id)
    
    tweet_url = f'https://twitter.com/{author_username}/status/{tweet_id}'
    #print(f'urls are {urls}')
    if len(urls) == 0:
        urls.append('NA')
    tweet_info = [create_date, create_time, tweet_url, tweet_id, author_username, urls[0], text ]
    
    return tweet_info
    
    
    
    

In [15]:
def scrub_text(t):
    
    if isinstance(t, str):

        #remove newlines
        t = re.sub('\n', ' ', t)

        #remove amp
        t = re.sub('&amp;', ' and ', t)

          #remove pipes because I use them as delimiters
        t = t.replace('|', ' ')  

        #remove unicode special chars
        string_encode = t.encode("ascii", "ignore")
        t = string_encode.decode()
    
    return t

assert scrub_text('asdfa|sdf &amp; dsdfgsdfg') == 'asdfa sdf  and  dsdfgsdfg'


In [ ]:
#override for testing
#queries = ['openstreetmap']
#stop_after = 1

i=0
while i != stop_after:
  
    i += 1
    total_result_count=0
    runDate = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    runDate_file_format = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
    queries = refresh_queries(control_input)
    for query in queries:
        for language in search_languages:
            print('\n\n')
            translation = translator.translate(query, src="en", dest=language)
            search = tweetQueryBuilder(translation.text)

            folder = 'test_twitter_data/staged_tweets/'

            #search = search + '&result_type=popular'
            #print(search)
            response = requests.get(search, headers={"Authorization":bearer})


            parsed = json.loads(response.text)

            # get results cout
            rc = parsed['meta']['result_count']
            print(f'{query} {language} yielded {rc} results.')
            
            #if at least one result was returned
            if rc > 0:
                
                
                
                ##check for hard dupe
                #df_already_saved_tweets = pd.DataFrame(tweet_list.get_all_records()) #seed the list with one row of data
                
                ##check for URL
                ##get author
                ##get content
                ##persist data
                
                
                
                print(f'attempting to write {rc} records to google sheets.')
                write_results_to_google_sheets(parsed) #export to google sheets
            
            total_result_count += rc

            #print(json.dumps(parsed, indent=4, sort_keys=True))

            fullFilePath = folder + runDate_file_format + '_' + query + '_' + language +'.json'
            #print(fullFilePath)
            print(f"{runDate} | {fullFilePath} | Record Count | {str(rc)}")

            with open(fullFilePath, 'w') as outfile:
                outfile.write(json.dumps(parsed, indent=4, sort_keys=True))
    print(f"{runDate} | | Total Results |  {total_result_count}")
    
    time.sleep(sleeptime)
    
    #time.sleep(24*60*60)
    




humanitarian response blockchain en yielded 0 results.
2022-11-13 12:46:15 | test_twitter_data/staged_tweets/2022-11-13_12-46-15_humanitarian response blockchain_en.json | Record Count | 0



humanitarian blockchain en yielded 7 results.
attempting to write 7 records to google sheets.
dupeurl https://odessa-journal.com/an-official-website-has-been-launched-to-collect-cryptocurrency-in-support-of-ukraine/
dupeurl https://buff.ly/3WLROFd
dupeurl http://MintMachine.Art
dupeurl https://www.binance.charity/
No new content to add
AT completion of write to google sheets, stats are 0 tweet id dupes, 6 no urls,            4 url dupes, 0 new
2022-11-13 12:46:15 | test_twitter_data/staged_tweets/2022-11-13_12-46-15_humanitarian blockchain_en.json | Record Count | 7



blockchain privacy risk en yielded 3 results.
attempting to write 3 records to google sheets.
dupeurl https://cryptoandcoin.news/ftx-blocks-aztec-network-privacy-dapp-calling-it-a-high-risk-mixer/?feed_id=37910&_unique_id=636b72f

humanitarian surge en yielded 4 results.
attempting to write 4 records to google sheets.
dupeurl https://on.cfr.org/3G5x1GJ
dupeurl https://www.euractiv.com/section/europe-s-east/news/migrants-rescued-from-swamp-on-polish-belarus-border-as-numbers-rise/
No new content to add
AT completion of write to google sheets, stats are 0 tweet id dupes, 4 no urls,            2 url dupes, 0 new
2022-11-13 12:46:15 | test_twitter_data/staged_tweets/2022-11-13_12-46-15_humanitarian surge_en.json | Record Count | 4



surge assessment en yielded 12 results.
attempting to write 12 records to google sheets.
dupeurl http://bit.ly/3hwjm11
dupeurl https://www.news4jax.com/news/local/2022/11/10/clay-county-significant-flooding-around-black-creek-doctors-inlet-due-to-storm-surge/?utm_source=twitter&utm_medium=social&utm_campaign=snd&utm_content=wjxt4
dupeurl http://dlvr.it/ScYZZq
dupeurl http://Nature.com
dupeurl http://Nature.com
dupeurl http://bit.ly/3DRqBbp
dupeurl https://cera.coastalrisk.live/
dupeurl 

disaster blockchain en yielded 37 results.
attempting to write 37 records to google sheets.
dupeurl https://www.forbes.com/sites/hershshefrin/2022/11/13/avoiding-the-next-ftx-type-disaster/
dupeurl https://vrdigitalminds.com/2022/11/blockchain-gaming-dapps-nonetheless-setting-up-no-matter-ftx-crypto-catastrophe-%e2%8b%86-cryptopreneurss/
dupeurl https://www.bocvip.com/265473/shocking-662m-theft-today-crypto-fraud-ftx-exposed/
dupeurl http://Blockchain.com
dupeurl https://blogo-news.com/ftx-bankruptcy-concerns/
dupeurl https://www.linkedin.com/posts/hitoshikokumai_democracy-privacy-ethics-activity-6995642013626880000-cik_
dupeurl https://dailycrypto.live/blockchain-gaming-dapps-nonetheless-constructing-regardless-of-ftx-crypto-disaster/
dupeurl https://news.cryptokingdomz.com/ftx-ftx-us-and-alameda-file-for-chapter-11-chapter/?feed_id=27303&_unique_id=636e7c7aab1d8
dupeurl https://news.cryptokingdomz.com/no-funds-from-tether-as-ftx-faces-9b-liquidity-disaster/?feed_id=27294&_unique_id=6

disaster openstreetmap en yielded 1 results.
attempting to write 1 records to google sheets.
dupeurl http://Disaster.Ninja
No new content to add
AT completion of write to google sheets, stats are 0 tweet id dupes, 0 no urls,            1 url dupes, 0 new
2022-11-13 12:46:15 | test_twitter_data/staged_tweets/2022-11-13_12-46-15_disaster openstreetmap_en.json | Record Count | 1
2022-11-13 12:46:15 | | Total Results |  342


# END